# Simulating Images with Nonnominal PSFs

For a significant part of the JWST commissioning process (until around MIMF-3, expected L+118 days), the JWST OTE will not be optimally aligned. The mirrors will be unstacked and unphased, and thus the nominal JWST PSFs that MIRaGe uses when creating imaging simulations do not adequately represent this type of telescope state.

In this notebook, we demonstrate how to use MIRaGe to simulate early commissioning images using nonnominal PSFs. The process is as follows:
- Parse APT output files to access the details and structure of a given program.
- Generate PSF libraries for the desired mirror state for each visit/exposure in a given observation, using `webbpsf` or existing FITS files.
- Generate MIRaGe YAML input files that include the directory in which to find the the nonnominal PSF libraries.
- Generate seed images from those YAML files that use the appropriate PSF library for the desired exposures. Follow the nominal procedures for adding dark exposure and detector effects.

### Table of Contents:
1. [Export program information from APT](#export_apt)
2. [Create diverse PSF library files](#create_psfs)
3. [Create `.yaml` files for each exposure](#make_yamls)
4. [Generate the simulated image](#simulate_images)

Appendix A: [Generating data for an entire observation](#simulate_whole_obs)

Appendix B: [Combining data into a mosaic](#mosaic)

## Import necessary packages and modules

In [ ]:
# Standard Library Imports
from glob import glob
import os
import pprint
import shutil
import time

# Third Party Imports
from astropy.io import fits
from matplotlib import cm
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np
import pysiaf
import webbpsf

# Local Imports (from nircam_simulator package)
from mirage import imaging_simulator
from mirage.apt import apt_inputs
from mirage.catalogs import get_catalog
from mirage.utils.utils import ensure_dir_exists
from mirage.yaml import yaml_generator

# View matplotlib plots inline
%matplotlib inline

---
<a id='export_apt'></a>
# 1. Export Program Information from APT

MIRaGe requires APT program output files in order to generate data with nonnominal PSFs.

### Get needed files from APT program

Open the APT file for the program you want to simulate. If you don't have the file locally, you can load this program in APT by selecting `File > Retrieve from STScI > Retrieve Using Proposal ID` and then entering the program ID (e.g. 1140). (You must be running APT in STScI mode for this retrieval method to be available.)

Export the `.pointing` and `.xml` files for your given proposal in APT by selecting `File > Export...` and selecting both the xml and pointing file options.

For this example, we will simulate images from OTE-17: Image Stacking 2. In this stage of commissioning, the 18 mirror segments are being moved to transition from a hexagonal image array to 17 stacked segments with one kicked out. We will only look at observation 1, for brevity's sake. The neccessary files, `OTE17-1153-obs1only.pointing` and `OTE17-1153-obs1only.xml`, are located within the `examples/nonnominal_psf_data/` directory.

In [ ]:
# Define the proposal ID
prop_id = 1153

# Where the pointing and XML file for this particular OTE CAR are located
input_dir = './nonnominal_psf_data/'

# Change the root if you named your files differently.
root = 'OTE17-{}-obs1only'.format(prop_id)
pointing_file = os.path.join(input_dir, '{}.pointing'.format(root))
xml_file = os.path.join(input_dir, '{}.xml'.format(root))

### Define location of output files

The process of generating simulated images with MIRaGe produces a lot of files:
- YAML files carrying the OTE mirror state
- YAML files carrying the specifications for simulations
- FITS files of the simulated seed, dark, and compiled images

Additionally, we must create FITS library files of the segment PSF images in order to simulate images with nonnominal PSFs.

Let's define the directories to save these output files to:

In [ ]:
# Where to save MIRaGe output
out_dir = './nonnominal_psf_data/output/'

# Where the segment PSF library files will be saved to (and later read from)
library_dir = os.path.join(out_dir, 'gridded_psf_library')

# Make sure both these directories exist
for full_path in [out_dir, library_dir]:
    ensure_dir_exists(full_path)

### Parse the `.pointing` file to get the number of PSF libraries needed

As mentioned above, we'll use the infrastructure of observation 1 in OTE-17: Image Stacking 2 (program 1153) for this example. 

This observation is a WFSC Commissioning observation that includes 17 Wavefront Control (WFC) groups to bring the mirrors in. (17 sets of image-mirror move-image plus 1 additional image.)

So, in order to generate the YAML configuration files for program 1153 observation 1, MIRaGe will need 18 different PSF libraries representing the 18 different mirror states across OTE-17 Observation 1. 

We can parse the `.pointing` file to verify this number:

In [ ]:
# Get the information from the pointing file
apt_prop = apt_inputs.AptInput()
pointing_tab = apt_prop.get_pointing_info(pointing_file, '1153')
n_exposures = len(pointing_tab['visit_id'])
print('MIRaGe requires PSFs for {} exposures.'.format(len(pointing_tab['visit_id'])))

---
<a id='create_psfs'></a>
# 2. Create unique, nonnominal PSF library files

Here we will outline two methods you can use to create an image with MIRaGe using unique PSFs for each exposure:
- If you have a list of **mirror moves/positions** describing different telescope states throughout different observations/visits, use [option 1](#adjustable_ote). We will use `webbpsf` to generate PSFs and then gridded PSF library objects from those telescope states.
- If you have a list of pre-existing **FITS files** with PSFs from different telescope states throughout different observations/visits, use [option 2](#to_model).

### Organizing PSF files

For every exposure simulation, MIRaGe will use the PSF library defined by the `psf_path` argument of that exposure's YAML file. So, to create a MIRaGe simulation with a nonnominal PSF, you must provide a path other than the default to the `psf_path` argument.

Thus, the easiest way to specify 18 different PSFs to be used for 18 different exposures is to:
1. Save each unique PSF library in a different directory
2. Pass an ordered list of the 18 separate directories to the YAML generator

There are a number of ways you could do the first step above, but here we choose to:
1. Generate a nested directory structure that matches the program structure
2. Save each PSF into the corresponding directory (i.e. `Observation001/Visit002/Activity03/PSFLibrary.fits`)

In [ ]:
# Create dictionary that mirrors the program structure
program_structure = {}
for i in range(n_exposures):
    obs_num = pointing_tab['obs_num'][i]
    visit_num = pointing_tab['visit_num'][i]
    activity_id = pointing_tab['act_id'][i]
    
    obs_key = 'Observation{}'.format(obs_num)
    visit_key = 'Visit{}'.format(visit_num)
    
    program_structure.setdefault(obs_key, {})
    visit_dict = program_structure[obs_key].setdefault(visit_key, []).append('Activity{}'.format(activity_id))                                                                      
    
pprint.pprint(program_structure)  

In [ ]:
# Create directory structure based on dictionary
psf_paths = []
program_dir = os.path.join(library_dir, root)
for observation in program_structure.keys():
    for visit in program_structure[observation].keys():
        for activity in program_structure[observation][visit]:
            activity_dir = os.path.join(program_dir, observation, visit, activity)
            ensure_dir_exists(activity_dir)
            psf_paths.append(activity_dir)
            
pprint.pprint(glob(program_dir + '/**/', recursive=True))

<a id='adjustable_ote'></a>
### Option 1: Use `webbpsf.enable_adjustable_ote()` to make a different PSF for each exposure

In this case, we use `webbpsf` to simulate 18 PSFs, 1 for each exposure, each of which have their own unique mirror state.

We will start with a image array (downgrading from large to medium for computational time), not yet coarsely phased. Then, we will bring one mirror segment in at a time for each of the 18 exposures.

In [ ]:
# Generate starting PSF: a large image array with random pistons.

# Initialize the webbpsf NIRCam object and turn on adjustable mirrors
nc = webbpsf.NIRCam()
nc, ote = webbpsf.enable_adjustable_ote(nc)

# Set up a large image array with random pistons.
webbpsf.opds.setup_image_array(ote, reset=True, verbose=False, size='medium')

# Add random pistons
random_pistons = np.random.randn(18)*20  # substantial coarse phasing erorrs. 
for i, seg in enumerate(ote.segnames[0:18]):  # don't piston "segment 19" the SM
    ote.move_seg_local(seg, piston=random_pistons[i])
    
# Define the image size (here, 1024 x 1024 pixels)
fov_pixels = 1024

# Generate the PSF
# NOTE: we are choosing a polychromatic simulation here to better represent the
# complexity of simulating unstacked PSFs. See the WebbPSF website for more details.
original_psf = nc.calc_psf(nlambda=10, oversample=1, 
                           fov_pixels=fov_pixels, add_distortion=False)

In [ ]:
# Display the starting PSF
plt.imshow(original_psf[1].data, norm=LogNorm(), clim=(1e-10, 1e-5))
plt.show()

In [ ]:
# Calculate the mirror moves to bring the segments back in
tilt_correction = []
for i, segment in enumerate(ote.segment_state):
    xtilt = segment[0]
    ytilt = segment[1]
    tilt_correction.append((i, -xtilt, -ytilt))
pprint.pprint(tilt_correction)

In [ ]:
def create_psf_lib_for_ote(nc, ote, i_exposure, pointing_table, psf_paths, 
                           tilt, fov_pixels):
    """For a given exposure in an APT program, perturb an OTE state 
    according to the corresponding X & Y tilt, and generate 
    and save a gridded PSF library using that perturbed mirror state.
    
    Parameters
    ----------
    nc : webbpsf.webbpsf_core.NIRCam instance
        WebbPSF NIRCam instrument instance
    ote : webbpsf.opds.OTE_Linear_Model_WSS object
        Starting mirror state
    i_exposure : int
        The index of the exposure (starting at 0)
    pointing_tab : dict
        Dictionary containing information parsed from APT .pointing file
    psf_paths : list
        List of paths to the directories where each of the PSF libraries 
        (one per exposure) will be saved
    tilt : list
        List of tuples containing the X and Y tilts to apply to the 
        mirror state for each exposure
    fov_pixels : int
        The size of one side of the square output PSF library, in pixels
    """
    start_time = time.time()
    psf_dir = psf_paths[i_exposure]
    
    print('Calculating PSF for obs {}, visit {}, exposure {}'
          .format(pointing_table['obs_num'][i_exposure], 
                  pointing_table['visit_num'][i_exposure], 
                  i_exposure + 1))

    i_seg, x_tilt_corr, y_tilt_corr = tilt[i_exposure]
    ote.move_seg_local(ote.segnames[i_seg], xtilt=x_tilt_corr, ytilt=y_tilt_corr)
    nc.pupil = ote
    
    # Define the filter
    nc.filter = 'F212N'
    nc.detector = 'NRCA3'
    
    # Write out file
    filename = 'nircam_fovp{}_samp1_npsf1.fits'.format(fov_pixels)
    outfile = os.path.abspath(os.path.join(psf_dir, filename))
    print(outfile)

    # Generate the PSF grid
    grid = nc.psf_grid(num_psfs=1, save=True, all_detectors=False,
                       use_detsampled_psf=True, fov_pixels=fov_pixels,
                       oversample=1, outfile=outfile, add_distortion=False,
                       nlambda=10)
    
    print('Elapsed time: {}\n'.format(time.time() - start_time))

<div class="alert alert-block alert-warning">
    
<b>Warning:</b> <br>

The for loop below will take a while: about 5 seconds for a 1024x1024 array, per detector, per filter! We've written it so that it will only write out for NIRCam A3 with the F212N filter, but know that if you need to generate more PSFs for different telescope states, detectors, and filters, this simulation might take quite a while.

</div>

In [ ]:
# Run create_psf_lib_for_ote() for each of the 18 exposures in OTE-17 Observation 1
for i_exposure in range(n_exposures):
    create_psf_lib_for_ote(nc, ote, i_exposure, pointing_tab, psf_paths, 
                           tilt_correction, fov_pixels)

<a id='to_model'></a>
### Option 2: Use pre-existing FITS file as PSF for each visit

In this case, we assume you have pre-existing PSF fits files stored somewhere (e.g. ITM simulations) that you want to MIRaGe to use.

(To be clear, the process below is not a continuation of [Option 1](#adjustable_ote), but a separate process that would be run instead.)

Thus, we must copy these pre-existing FITS files into the program directory structure.

In [ ]:
# Note: you must have access to the STScI central storage directories to use these data.
psf_fits_data_dir = '/grp/jwst/ote/mirage_example_data/'
all_ote17_fits = sorted(glob(psf_fits_data_dir + '*.fits'))
all_ote17_fits

In [ ]:
# Just for fun, plot all the data
fig, axes = plt.subplots(3, 6, figsize=(15, 8))
plt.tight_layout()

i = 1
for ax, f in zip(axes.flatten(), all_ote17_fits):
    with fits.open(f) as hdulist:
        psf_data = hdulist[1].data
    ax.imshow(psf_data, clim=(0.1, 100))
    ax.set_title('Exposure {}'.format(i))
    i += 1
        
plt.show()

In [ ]:
# Copy the images into the observation/visit/activity-specific directories
for psf_path, fits_file in zip(psf_paths, all_ote17_fits):
    shutil.copy(fits_file, psf_path)
    print('Copied PSF to: {}'.format(os.path.abspath(os.path.join(psf_path, os.path.basename(fits_file)))))
    print()

---
<a id='make_yamls'></a>
# 3. Create YAML files for each exposure

Next, we need to make the YAML files that include all of the parameters for MIRaGe to run.

Here we will provide the MIRaGe YAML generator with an ordered list of each PSF path we filled above, i.e.: 

    [Observation001/Visit001/Activity01/', 'Observation001/Visit001/Activity02/', ...]

### Query online catalogs to generate catalog files of sources around the target star

Next, we need to generate catalog files containing RA, Dec, and magnitude for the sources around the target (or targets) in this proposal. 

First we must parse the `.pointing` file to determine the RA and Dec of the target (or targets) that will be observed. Then we will query the appropriate star catalogs to get the magnitudes and locations of shortwave and longwave sources, respectively, around the target(s). If you are using F212N and F480M NIRCam filters, you can use the 2MASS and WISE catalogs, respectively. If you are using one of these two filters, all of this can be accomplished with the `mirage.get_catalog.get_all_catalogs` function.  

If different observations within the proposal have different targets, separate catalogs will be made for each target.

These catalog files will be written to your local `mirage/catalogs/` directory. If files for a given catalog and RA/Dec have already been generated, they will not be regenerated.

<div class="alert alert-block alert-warning">
    
<b>Important:</b> <br>

Querying 2MASS and WISE is only appropriate for observations with the F212N and F480M NIRCam filters. If you want to simulate observations that use other filters, you will have to either query different bandpasses or catalogs or perform a photometric conversion on an existing catalog.

</div>

In [ ]:
# Generate and save shortwave and longwave individual catalogs for the target stars
cats = get_catalog.get_all_catalogs(pointing_file, prop_id)
target_coords, catalog_filenames_sw, catalog_filenames_lw = cats

cat_dict = {'nircam': {'lw': catalog_filenames_lw,
                       'sw': catalog_filenames_sw}}

### Generate the `.yaml` files

MIRaGe's YAML generator,`mirage.yaml.yaml_generator`, will create all of the YAML files for a whole APT program at once - one YAML file per exposure in the program.

**Some additional settings are required to ensure `yaml_generator` works for nonnominal PSF simulations.** You must specify theses attribute before running `create_inputs()` to make the YAML files correctly:
- `yam.psf_paths = [list or str]` - tells MIRaGe to use files in the given directory/directories for the PSFs paths for different exposures. `psf_paths` can be defined as:
    - *A string pointing to a directory* - the matching PSF library in that directory will be written into ALL YAMLs, and thus will be used for ALL exposures in the program
    - *A list of strings pointing to a list of directories* - Each directory in the list will be mapped onto the corresponding YAML, in chronological order: the 0th listed directory will be used for the 0th exposure/YAML, the 1st listed directory for the 1st exposure/YAML, and so on. Thus each exposure will be simulated using its respective, potentially different, PSF library. The list must be the same length as the number of exposures in the program.
    <br><br>
- `yam.add_psf_wings = False` - tells MIRaGe not to add wings to the PSF. We don't need wings in this case, since our PSFs are so large.
<br><br>

In [ ]:
# Create all input YAML files (one per each exposure)
yaml_dir = os.path.join(out_dir, 'yamls')
yam = yaml_generator.SimInput(input_xml=xml_file, pointing_file=pointing_file,
                              catalogs=cat_dict,
                              verbose=True, output_dir=yaml_dir, simdata_output_dir=out_dir)
yam.psf_paths = psf_paths
yam.add_psf_wings = False
yam.create_inputs()

## Choose which exposure to simulate

Now that we've generated all of the needed YAML files, we need to choose one to simulate images with. MIRaGE can only generate one simulated exposure at a time, so we need to choose one YAML file in our `yamls` directory that we will use to produce an image. (See [Appendix A](#simulate_whole_obs) for how use a wrapper to simulate multiple exposures at once with MIRaGe.)

Not every exposure necessarily has the same pointing, so we should choose an exposure that places the target star in the desired detector field-of-view.

### Examine target pointings relative to apertures and V2/V3 references

Looking at the `.pointing` file, let's plot where the target will appear relative to the NIRCam apertures for each unique pointing.

We'll also print out the first YAML in the exposure list for each pointing. In this case, this just means printing out the first YAML, since there is only one pointing across all the exposures. However, if you are plotting the pointings for an observation that contains more than one pointing, then another YAML file will be added to the "Example files for each pointing:" list, corresponding to the exposure when the pointing changes.

In [ ]:
# Examine apertures and V2/V3 references for each array/subarray
nc_siaf = pysiaf.Siaf('NIRCam')
nc_full = nc_siaf['NRCA1_FULL']

plt.figure(figsize=(15,10))
for apername in sorted(nc_siaf.apernames):
    a = apername
    if ('_FULL' in a) and ('OSS' not in a) and ('MASK' not in a) and (a[-1] != 'P'):
        nc_siaf[a].plot(frame='tel', label=a, fill_color='white')

# Compare V2/V3 of targets (from .pointing file)
all_pointings = set([(v2, v3, filename) for v2, v3, filename in zip(yam.info['v2'], 
                                                                yam.info['v3'], 
                                                                yam.info['yamlfile'])])

print('Example files for each pointing:')
print('--------------------------------')
plotted_points = []
for i_point, (v2, v3, filename) in enumerate(all_pointings):
    if (v2, v3) not in plotted_points:
        plotted_points.append((v2, v3))
        plt.scatter(v2, v3, marker='*', s=500, 
                    label='Pointing {}/{}'.format(i_point + 1, len(all_pointings)))
        print('{}. {}'.format(i_point + 1, filename))

plt.legend()

plt.show()

This is easy for 1153 Obs. 1, since all 18 exposures have the same pointing. We'll just choose a YAML that has a detector and filter that matches the library files we have created so far - NRCA3 and F212N.

Let's choose `jw01153001001_01101_00001_nrca3.yaml`.

*See [JDox](https://jwst-docs.stsci.edu/display/JDAT/File+Naming+Conventions+and+Data+Products) for a detailed explanation of the MIRaGE YAML file name format.*

In [ ]:
# Select one YAML where the target will be in the field of view
yfiles = glob(os.path.join(yaml_dir, '*.yaml'))
file_name_to_match = 'jw01153001001_01101_00001_nrca3'
yaml_to_sim = os.path.abspath([y for y in yfiles if file_name_to_match in y][0])
print(yaml_to_sim)

---
<a id='simulate_images'></a>
# 4. Simulate image with MIRaGe

Finally, we can run MIRaGe to generate a seed image simulation of our unstacked mirror state during OTE-17.

From here on out, from the user perspective, the simulation process is identical to that of nominal imaging cases (see the [imaging example notebook](#Imaging_simulator_use_examples.ipynb). To reiterate, it is the specifications made in the YAML files that enable the simulation of nonnominal mirror simulations with MIRaGe.

In [ ]:
# Run the image simulator using the input defined in yaml_to_sim
img_sim = imaging_simulator.ImgSim()
img_sim.paramfile = yaml_to_sim
img_sim.create()

In [ ]:
# Plot the seed image, dark image, and final exposure simulation
fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=(20, 7))
plt.tight_layout()

# Define scale limits and colormap
clim = (1e-2, 1e4)
cmap = cm.get_cmap('viridis')
cmap.set_bad(cmap(0))

# Plot seed image
fitsplot = ax1.imshow(img_sim.seedimage, clim=clim, norm=LogNorm(), cmap=cmap)
ax1.set_title('Seed Image', size=24)

# Plot dark current
dark_diff = img_sim.linDark.data[0,-1,:,:] - img_sim.linDark.data[0,0,:,:]
ax2.imshow(dark_diff, 
           clim=clim, norm=LogNorm())
ax2.set_title('Dark Current', size=24)

# Plot final exposure
linear_output = './nonnominal_psf_data/output/jw01153001001_01101_00001_nrca3_linear.fits'
with fits.open(linear_output) as h:
    lindata = h[1].data
    header = h[0].header
exptime = header['EFFINTTM']
diffdata = (lindata[0,-1,:,:] - lindata[0,0,:,:]) / exptime

ax3.imshow(diffdata, clim=clim, norm=LogNorm())
ax3.set_title('Final Exposure Simulation', size=24)

# Define the colorbar
cbar_ax = fig.add_axes([1, 0.09, 0.03, 0.87])
cbar = plt.colorbar(fitsplot, cbar_ax)
cbar.set_label('Count Rate', rotation=270, labelpad=30, size=24)

---
<a id='simulate_whole_obs'></a>
# Appendix A: Simulating many exposures at once

Chances are, you don't want to simulate just one exposure from one detector. In order to simulate all of the exposures from a given observation, write a for loop to iterate over all the YAMLs. We include an example for program 1134 observation 1 below.

### 1. Create all PSF library files
First, make sure that you have created library files for all of the filters and detectors that will be simulated in your observation. (This might mean continuing [step 2](#create_psfs) above.)

### 2. Run `imaging_simulator` for all YAMLs
Second, grab all of the YAMLs for that observation and run the image simulator on them all.
```python
from glob import glob
import os

from mirage import imaging_simulator

# Get all the 1153 Obs 1 yamls
all_yaml_files = glob(os.path.join(yaml_dir, 'jw01153001*.yaml'))
n_yamls = len(all_yaml_files)
print('{} FITS files will be generated.'.format(n_yamls))

for yaml in all_yaml_files:
    print('*** SIMULATING YAML {}/{}: {} ***'.format(i+1, n_yamls, yaml))
    img_sim = imaging_simulator.ImgSim()
    img_sim.paramfile = yaml
    img_sim.create()
```

(If you are impatient and ambitious, you can use Python's `multiprocessing` module to the simulation go faster. Even better on a server with more processors!)

To learn how to combine multiple exposure simulations into a mosaic with QUIP, see [Appendix B](#mosaic).

---
<a id='mosaic'></a>
# Appendix B: Combine into a mosaic

The [`wss_tools`](https://wss-tools.readthedocs.io/en/latest/) include a software program, QUIP, which can be used to combine a list of FITS files into a single mosaic image. QUIP requires an operations file, which we describe how to make here.

### Turn linear FITS products into slope images
```python
from glob import glob
import os

from astropy.io import fits

from mirage.utils.utils import ensure_dir_exists

obs1_fits = glob(os.path.join(out_dir, 'jw*linear.fits'))
print('{} FITS files produced for program APT 1153 Observation 1'.format(len(obs1_fits)))

# Subtract the first from last for each ramp
for f in obs1_fits:
    with fits.open(f) as hdulist:
        data = hdulist[1].data
        hdr = hdulist[1].header
        
    diff = data[0, -1] - data[0, 0]

    hdu = fits.PrimaryHDU(data=diff, header=hdr)

    new_filename = os.path.join(out_dir, 'slope_fits', os.path.basename(f))
    ensure_dir_exists(os.path.dirname(new_filename))
    hdu.writeto(new_filename, overwrite=True)
    
obs1_slope_fits = glob(os.path.join(out_dir, 'slope_fits', 'jw*linear.fits'))
```


### Make ops file for QUIP
```python
# Set variables for writing QUIP ops file
quip_dir = os.path.join(out_dir, 'quip')
ensure_dir_exists(quip_dir)
outfile = 'congrid'
bindim = 2048
opsfile = os.path.join(quip_dir, 'ops_file_'+outfile.strip("/")+str(bindim)+'.xml')

# Write the file
f = open(opsfile, 'w')

f.write('<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n')
f.write('<QUIP_OPERATION_FILE xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" creator="WSS Executive" time="16:22:40.093Z" date="2017-06-14Z" version="6.0.1" operational="false" xsi:noNamespaceSchemaLocation="/Users/lajoie/TEL/WSS-6.0.1/Software/schema/quip_operation_file.xsd">\n')
f.write('    <CORRECTION_ID>R2017061401</CORRECTION_ID>\n')
f.write('    <OPERATION_TYPE>THUMBNAIL</OPERATION_TYPE>\n')
f.write('    <IMAGES>\n')

for filename in obs1_slope_fits:
    f.write("       <IMAGE_PATH>{:s}</IMAGE_PATH>\n".format(filename))
    
f.write( '       </IMAGES>\n'    )
f.write( '       <OUTPUT>\n')
f.write( '           <OUTPUT_DIRECTORY>{:s}quip/</OUTPUT_DIRECTORY>\n'.format(quip_dir))
f.write( '           <LOG_FILE_PATH>{:s}quip/R2017061401_quip_activity_log.xml</LOG_FILE_PATH>\n'.format(quip_dir))
f.write( '           <OUT_FILE_PATH>{:s}quip/R2017061401_quip_out.xml</OUT_FILE_PATH>\n'.format(quip_dir))
f.write( '       </OUTPUT>\n')

f.write('</QUIP_OPERATION_FILE>\n')

f.close()

print('Successfully wrote QUIP ops file to', opsfile)
```